## 문제 2-1 : 콤마 구분 리스트 파서 활용
### 출력 파서: CommaSeparatedListOutputParser
### 문제
사용자가 관심있는 분야(예: "음식", "스포츠", "영화" 등)를 입력하면, 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 출력하는 프로그램을 작성하세요.
### 요구사항:
- CommaSeparatedListOutputParser 사용
- 사용자 입력을 받아 동적으로 처리
- 한국 관련 내용으로 추천
### 예시 입력: "음식"
### 예시 출력: 
['명동 칼국수', '부산 돼지국밥', '전주 비빔밥', '춘천 닭갈비', '제주 흑돼지']


In [4]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI 
import os
from dotenv import load_dotenv

## 기본 세팅. llm, parser
load_dotenv()

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
output_parser = StrOutputParser()
comma_parser = CommaSeparatedListOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 한국 문화 전문가야. 사용자가 입력한 분야와 관련된 한국의 유명한 장소나 활동 5가지만 출력해. 다른 말 붙이지 말고"),
    ('human', "user_interest"),
    ("ai", "{format_instructions}")
])

user_interest = input("관심 있는 분야를 입력하세요 > ")

chain = prompt | llm | comma_parser
response = chain.invoke(user_interest)
print(response)


['* 서울의 N서울타워 ', '* 광주의 5.18 기념관 ', '* 부산의 자갈치시장 ', '* 문의문화재단지 ', '* 경주의 황남빵집']


## 문제 2-2 : 영화 리뷰 감정 분석기
### 출력 파서: EnumOutputParser
### 문제
영화 리뷰 텍스트를 입력받아 감정을 "긍정", "부정", "보통" 중 하나로 분류하는 시스템을 만드세요.
### 요구사항:
- EnumOutputParser와 Enum 클래스 사용
- 여러 개의 테스트 리뷰로 검증
- 감정 분류 결과를 깔끔하게 출력
### 테스트 리뷰 예시:
- "이 영화 정말 재미없어요. 시간 낭비였습니다."
- "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
- "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."

- 분석실패했을 경우에 사용하는 
OutputFixingParser 와 EnumOutputParser 를 같이 사용하는 것은 선택입니다.


In [6]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

class Sentiment(Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"
    
enum_parser = EnumOutputParser(enum=Sentiment)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 영화 리뷰 감정 분석 전문가야. 주어진 영화 리뷰의 감정을 '긍정', '부정', '보통' 중 하나로만 분류해. 다른 설명 없이 감정 단어 하나만 출력해야 해."),
    ("human", "영화 리뷰: {review}\n{format_instructions}") # 파서가 필요한 출력 형식을 삽입할 Placeholder
])

sentiment_chain = prompt.partial(format_instructions=enum_parser.get_format_instructions()) | llm | enum_parser
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

for i, review in enumerate(test_reviews):
    print(f"\n[리뷰 {i+1}]")
    print(f"리뷰 텍스트: \"{review}\"")
    sentiment = sentiment_chain.invoke({"review": review})
    print(f"분석된 감정: {sentiment.value}") # Enum 값(예: "긍정")을 출력



[리뷰 1]
리뷰 텍스트: "이 영화 정말 재미없어요. 시간 낭비였습니다."
분석된 감정: 부정

[리뷰 2]
리뷰 텍스트: "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
분석된 감정: 긍정

[리뷰 3]
리뷰 텍스트: "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
분석된 감정: 보통


## 문제 2-3: 학생 정보 구조화 시스템
### 출력 파서: PydanticOutputParser
### 문제
학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 추출하는 시스템을 만드세요.
### 요구사항:
- PydanticOutputParser와 BaseModel 사용
- 각 필드에 적절한 타입과 설명 추가
- 자유 형식의 텍스트에서 정보 추출
### 예시 입력:
 "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
### 예시 출력 구조:
{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": ["게임하기", "영화보기", "코딩"],
    "goal": "훌륭한 개발자가 되는 것"
}


In [10]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import PydanticOutputParser
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 목록")
    goal: str = Field(description="학생의 장래 목표")


pydantic_parser = PydanticOutputParser(pydantic_object=StudentInfo)
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 학생 자기소개에서 정보를 추출하는 전문가야. 다음 자기소개에서 학생의 이름, 나이, 전공, 취미 리스트, 목표를 정확하게 추출하여 JSON 형식으로 구조화해줘.\n{format_instructions}"),
    ("human", "학생 자기소개: {introduction}"),
])

info_chain = prompt.partial(format_instructions=pydantic_parser.get_format_instructions()) | llm | pydantic_parser
test_introduction = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

info_response = info_chain.invoke(test_introduction)
print(info_response)
print(type(info_response))


name='김민수' age=22 major='컴퓨터공학' hobbies=['게임하기', '영화보기', '코딩'] goal='훌륭한 개발자가 되는 것'
<class '__main__.StudentInfo'>


## 문제 2-4 : 여행 계획 분석기
### 출력 파서: StructuredOutputParser
### 문제
여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.
### 요구사항:
- StructuredOutputParser와 ResponseSchema 사용
- 5개의 필드를 정의하여 정보 추출
- 자연어 텍스트에서 핵심 정보 파싱
### 예시 입력:
 "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
### 예시 출력 구조:
{
    "destination": "부산",
    "duration": "2박 3일",
    "budget": "30만원",
    "rating": "4",
    "activities": ["해운대 바다구경", "자갈치시장 회 먹기", "감천문화마을 구경"]
}

In [5]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from typing import List

# 추출할 정보의 스키마 정의
response_schemas = [
    ResponseSchema(name="destination", description="여행을 다녀온/갈 장소"),
    ResponseSchema(name="duration", description="여행 기간 (예: '2박 3일', '5일')"),
    ResponseSchema(name="budget", description="여행에 사용된/예상되는 총 예산 (예: '30만원', '500달러')"),
    ResponseSchema(name="rating", description="여행에 대한 추천도 (1점에서 5점 사이)"),
    ResponseSchema(name="activities", description="여행 중 했던/할 주요 활동 목록")
]

struct_parser = StructuredOutputParser.from_response_schemas(response_schemas)
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 여행 후기 분석 전문가야. 주어진 여행 텍스트에서 다음 정보를 정확하게 추출하여 JSON 형식으로 구조화해줘.\n{format_instructions}"),
    ("human", "여행 텍스트: {travel_text}"),
])

test_travel_text =  "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
travel_analyzer_chain = prompt.partial(format_instructions=struct_parser.get_format_instructions()) | llm | struct_parser
travel_response = travel_analyzer_chain.invoke(test_travel_text)
print(travel_response)


{'destination': '부산', 'duration': '2박3일', 'budget': '30만원', 'rating': '4점', 'activities': '해운대 바다구경, 자갈치시장에서 회 먹기, 감천문화마을 구경'}
